In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
tf.keras.utils.to_categorical

<function tensorflow.python.keras.utils.np_utils.to_categorical(y, num_classes=None, dtype='float32')>

In [2]:
train_data = pd.read_csv('classification.csv', dtype=str)
#data = pd.read_csv('./Hand_Annotations_2.csv',dtype=str,names=colnames
Y = train_data[['label']]

kf = KFold(n_splits = 10)
                         
skf = StratifiedKFold(n_splits = 10, random_state = 7, shuffle = True)
image_shape = (150,150,3)

In [3]:
idg = ImageDataGenerator(#width_shift_range=0.1,
                         #height_shift_range=0.1,
                         #zoom_range=0.3,
                         fill_mode='nearest',
                         horizontal_flip = True)
                         #rescale=1./255)

In [4]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [5]:
#model = tf.keras.models.Sequential()
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu'))

# Dropouts help reduce overfitting by randomly turning neurons off during training.
# Here we say randomly turn off 50% of neurons.
model.add(Dropout(0.5))

# Last layer, not binary!
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy', #not binary for the poses
              optimizer='rmsprop',
              metrics=['accuracy'])

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

save_dir = 'kfoldModels/'
fold_var = 1

for train_index, val_index in kf.split(np.zeros(875),Y):
    training_data = train_data.iloc[train_index]
    validation_data = train_data.iloc[val_index]

    train_data_generator = idg.flow_from_dataframe(training_data, target_size=image_shape[:2],
                               batch_size = 16,
                               x_col = "filename", y_col = "label",
                               class_mode = "categorical", shuffle = True)
    valid_data_generator  = idg.flow_from_dataframe(validation_data, target_size=image_shape[:2],
                            batch_size = 16,
                            x_col = "filename", y_col = "label",
                            class_mode = "categorical", shuffle = True)
    
  

    # CREATE NEW MODEL
    #model = create_new_model()

    # COMPILE NEW MODEL
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

    # CREATE CALLBACKS
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
                            monitor='val_accuracy', verbose=1, 
                            save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    # There can be other callbacks, but just showing one because it involves the model name
    # This saves the best model
    # FIT THE MODEL
    history = model.fit(train_data_generator,
                epochs=10,
                #steps_per_epoch=150,
                callbacks=callbacks_list,
                validation_data=valid_data_generator)
    #PLOT HISTORY

    # LOAD BEST MODEL to evaluate the performance of the model
    model.load_weights("kfoldModels/model_"+str(fold_var)+".h5")

    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names,results))

    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])

    tf.keras.backend.clear_session()

    fold_var += 1



Found 787 validated image filenames belonging to 3 classes.
Found 88 validated image filenames belonging to 3 classes.
Epoch 1/10
50/50 [==============================] - 10s 182ms/step - loss: 27.4619 - accuracy: 0.6468 - val_loss: 0.4139 - val_accuracy: 0.8295

Epoch 00001: val_accuracy improved from -inf to 0.82955, saving model to kfoldModels/model_1.h5
Epoch 2/10
50/50 [==============================] - 9s 177ms/step - loss: 0.7604 - accuracy: 0.7662 - val_loss: 0.9851 - val_accuracy: 0.7955

Epoch 00002: val_accuracy did not improve from 0.82955
Epoch 3/10
50/50 [==============================] - 9s 179ms/step - loss: 0.8452 - accuracy: 0.8424 - val_loss: 0.5695 - val_accuracy: 0.8636

Epoch 00003: val_accuracy improved from 0.82955 to 0.86364, saving model to kfoldModels/model_1.h5
Epoch 4/10
50/50 [==============================] - 9s 179ms/step - loss: 0.3474 - accuracy: 0.8844 - val_loss: 0.5253 - val_accuracy: 0.8182

Epoch 00004: val_accuracy did not improve from 0.86364
Ep


Epoch 00006: val_accuracy did not improve from 0.98864
Epoch 7/10
50/50 [==============================] - 9s 179ms/step - loss: 0.0257 - accuracy: 0.9924 - val_loss: 1.1626 - val_accuracy: 0.8409

Epoch 00007: val_accuracy did not improve from 0.98864
Epoch 8/10
50/50 [==============================] - 9s 179ms/step - loss: 0.0563 - accuracy: 0.9936 - val_loss: 0.0859 - val_accuracy: 0.9886

Epoch 00008: val_accuracy did not improve from 0.98864
Epoch 9/10
50/50 [==============================] - 9s 178ms/step - loss: 0.0198 - accuracy: 0.9949 - val_loss: 0.1687 - val_accuracy: 0.9773

Epoch 00009: val_accuracy did not improve from 0.98864
Epoch 10/10
50/50 [==============================] - 9s 178ms/step - loss: 0.0155 - accuracy: 0.9962 - val_loss: 0.1585 - val_accuracy: 0.9659

Epoch 00010: val_accuracy did not improve from 0.98864
6/6 [==============================] - 0s 37ms/step - loss: 0.0079 - accuracy: 1.0000
Found 787 validated image filenames belonging to 3 classes.
Found

50/50 [==============================] - 9s 179ms/step - loss: 0.0237 - accuracy: 0.9898 - val_loss: 0.0575 - val_accuracy: 0.9885

Epoch 00002: val_accuracy did not improve from 1.00000
Epoch 3/10
50/50 [==============================] - 9s 177ms/step - loss: 0.0300 - accuracy: 0.9949 - val_loss: 1.0003e-07 - val_accuracy: 1.0000

Epoch 00003: val_accuracy did not improve from 1.00000
Epoch 4/10
50/50 [==============================] - 10s 191ms/step - loss: 0.0475 - accuracy: 0.9886 - val_loss: 7.2071e-07 - val_accuracy: 1.0000

Epoch 00004: val_accuracy did not improve from 1.00000
Epoch 5/10
50/50 [==============================] - 10s 195ms/step - loss: 0.0403 - accuracy: 0.9886 - val_loss: 1.3428e-07 - val_accuracy: 1.0000

Epoch 00005: val_accuracy did not improve from 1.00000
Epoch 6/10
50/50 [==============================] - 10s 196ms/step - loss: 0.0255 - accuracy: 0.9924 - val_loss: 9.3549e-04 - val_accuracy: 1.0000

Epoch 00006: val_accuracy did not improve from 1.00000
Ep

**First 10-Fold Cross Validation Results:**

List of Accuracies of the 10 Models: [1.0, 0.98591548204422, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

Average of 99.8591548204422%

*This doesn't include the images I used for testing*

In [9]:
VALIDATION_ACCURACY

[0.9090909361839294,
 1.0,
 0.9545454382896423,
 1.0,
 0.9886363744735718,
 1.0,
 1.0,
 1.0,
 0.9885057210922241,
 1.0]

In [10]:
(sum(VALIDATION_ACCURACY)/10) * 100

98.40778470039368